In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip archive.zip

# What are autobattles and autochess games?
For more info visit: https://dotesports.com/news/what-is-an-autobattler \

Autobattlers are tactical strategy games with drafting elements from card games.

Matches feature eight players and take place across several rounds. Players fight each other in one-vs-one matches defined at random.

Players fight each other by placing a set number of units on a board as they wish. These units, or pieces, fight each other automatically when a round starts. The player who defeats their opponent’s whole squad is declared the winner of a round, and when the round is over, all units on both sides are reset to their previous position to get ready for the next round.

When you lose a round, you lose points represented by the health of your tactician, not your units. The more opposing pieces alive after a round ends, the greater the damage you’ll take. Players who reach zero health points are knocked out of the match regardless of which round they’re in. The last player with their tactician standing is declared the winner of an autobattler match.

TFT (Teamfight Tactics) is an autobattler based on the characters from League of Legends. Each character can be equip with items to increase its strength. Moreover, each character has an "Origin" and belongs to one or two "Classes". Origin and Class bonuses occur when enough characters of a given Origin or Class are in play; these bonuses further increase character strength. 

In [ ]:
# imports
import json
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

# Formatting for dataframe
pd.set_option('display.max_columns', None)  

# Loading the data
tft_df = pd.read_csv('TFT_Challenger_MatchData.csv')
tft_df[:8]

The column that we are most interested in is the champion column. You'll see here that each entry is a JSON object, where the keys are the champions and the values are JSON objects describing the items and the star of the user. Items and stars are used to make individual champions stronger. Items are gained by picking them up in certain rounds, while stars are gained by getting a certain number of a character ("star 2" requires three of the same "star 1" champions. "star 3" requires three of the same "star 2" champions. ). In our case, we were only concerned with the champions. The items are encoded as numbers and nowhere in the dataset do they describe what number corresponds to each item, so, even if we did run a market basket analysis on this, we be able to say what it means. Stars, on the other hand, are just more of the same champion and, in 99.9999999% of the cases, you will just want the highest star (so 9 of the same "star 1" characters). It should just be obvious that you want as many of a champion as you can get to hit star 3.


In [ ]:
tft_df["champion"][0]

We import the dataset for the Challenger division (the top 0.013% of ranked players) and look at the players who placed in the top 4. Here we find sets with support of at least 0.05 across the 10,000 matches in the dataset.

In [ ]:
# Only taking the top 4 comps per match (Top 4 was chosen because if you place in the top 4 your rank will go up)
tft_df = tft_df[tft_df['Ranked'] <= 4]

# Change the champion column into JSONs
tft_df['champion'] = tft_df['champion'].apply(lambda x : x.replace("'","\""))
tft_df['champion'] = tft_df['champion'].apply(json.loads)

# Take the keys of each JSON (i.e. the Champions of each comp)
champions = list(map(lambda x: list(x.keys()), tft_df.champion))

# Formatting apriori by one-hot encoding using mlxtend.preprocessing.TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(champions).transform(champions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Applying the apriori to find the frequency of champions
challenger = apriori(df, min_support=0.05, use_colnames=True)
challenger['length'] = challenger['itemsets'].apply(lambda x: len(x))
challenger

We import a mapping of the characters and their respective origins/classes.

In [ ]:
# Import the champion trait data
champs = pd.read_csv('champs.csv')
champs.set_index('Champion', inplace=True)

# Format the data to a dictionary from champion to traits
champion_combination_map = {}
for champion in champions:
  for c in champion:
    if c in champion_combination_map:
      continue
    champion_combination_map[c] = list(filter(lambda x: type(x) == str,champs.loc[c, :])) 

champion_combination_map

We then found the frequency of each champion and listed their traits.

In [ ]:
count = {}
for champion in champions:
  for c in champion:
    count[c] = count.get(c, 0) + 1
list(map(lambda x: [champion_combination_map[x[0]]]+[x], sorted(count.items(), key=lambda x: x[1], reverse=True)))

From our most frequent sets of champions, we calculate how many of each trait we have. 

In [ ]:
def count_combination(champions):
  count = {}
  for champion in champions:
    for comb in champion_combination_map[champion]:
      count[comb] = count.get(comb, 0) + 1
  return count

In [ ]:
challenger['combination'] = challenger['itemsets'].apply(count_combination)
challenger

# Analysis of the Challenger Data

We look at the most common teams (support >= 0.2) with 3 or more characters. We find that all of them are Brawlers; this indicates that the Brawler Class preforms very well; we may want to choose Brawlers if we want to place among the top four players.

In [ ]:
challenger[ (challenger['length'] >= 3) &
                   (challenger['support'] >= 0.2)]

We look at the most common teams (support >= 0.18) with 6 or more characters. Again, we find that the Brawler Class does very well. We also see that the Blaster Class is a powerful combination with the Brawler Class.

---



In [ ]:
challenger[ (challenger['length'] >= 6) &
                   (challenger['support'] >= 0.18)]

We look at the most common teams (support >= 0.10) with 8 or more characters. Once again, we find that the Brawler and Blaster Classes make for a great combination, and actually see now that the larger team size has been used to add more Brawlers.

In [ ]:
challenger[ (challenger['length'] >= 8) &
                   (challenger['support'] >= 0.1)]

We decrease the support to find more teams with at least 8 characters. We see some other viable champions (AurelionSol) and see, again, that the Brawler Class is very effective.

In [ ]:
challenger[ (challenger['length'] >= 8) &
                   (challenger['support'] >= 0.05)]

We see that the most common comps that we found are actually legitimate compositions. For example: 


*   1098 = 6 Star Guardians
*   1099 = Mech-Pilot (Fizz, Annie, Rumble) Infiltrator (Fizz, Kaisa, Shaco)






In [ ]:
challenger[ (challenger['length'] == 6) &
                   (challenger['support'] >= 0.05) &
                   (list(map(lambda x: True if 'Vi' not in x and 'Malphite' not in x and 'Blitzcrank' not in x else False, challenger['itemsets'])))
                 ]

# Recommendations

Now what we wanted to do is, given a current set of champions, what are the successful comps that we can build out of it.  This function recommmends the next champions based off of confidence, lift, and support.

In [ ]:
def get_itemsets(frequent_itemsets, champions):
  champions = set(champions)
  row = frequent_itemsets[frequent_itemsets['itemsets'] == champions]
  df = frequent_itemsets[
                   (list(map(lambda x: champions.issubset(x) and x != champions, frequent_itemsets['itemsets'])))
                 ].sort_values(by='support', ascending=False)
  
  df['diffs'] = [frequent_itemsets[frequent_itemsets['itemsets'] == diffs].values[0][0] for diffs in (df['itemsets'] - champions)]
  df['confidence'] = df['support']/row['support'].values[0]
  df['lift'] = df['support']/row['support'].values[0]/df['diffs']
  df['combination'] = df['itemsets'].apply(count_combination)
  return df.loc[:, ['itemsets', 'length','support', 'confidence', 'lift', "combination"]]

Challenger recommendations for Vi, Shen, and Ekko

In [ ]:
get_itemsets(challenger, ['Vi', 'Shen', 'Ekko'])

Challenger Recommendations for Fizz, Annie, and Rumble

In [ ]:
get_itemsets(challenger, ['Fizz', 'Annie', 'Rumble'])

Challenger Recommendations for Annie

In [ ]:
get_itemsets(challenger, ['Annie'])

# Analysis of the Platinum Data

We analyzed the data from a lower rank to see any discrepancies (top 17.912% of players)


In [ ]:
# Read the data and take the top 4
tft_df = pd.read_csv('TFT_Platinum_MatchData.csv')
tft_df = tft_df[tft_df['Ranked'] <= 4]

# Formatting the data
tft_df['champion'] = tft_df['champion'].apply(lambda x : x.replace("'","\""))
tft_df['champion'] = tft_df['champion'].apply(json.loads)
champions = list(map(lambda x: list(x.keys()), tft_df.champion))

# One hot encoding
te = TransactionEncoder()
te_ary = te.fit(champions).transform(champions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Applying Apriori
plat = apriori(df, min_support=0.05, use_colnames=True)
plat['length'] = plat['itemsets'].apply(lambda x: len(x))

Teams with Fizz, Annie, and Rumble in the Platinum divison.

In [ ]:
get_itemsets(plat, ['Fizz', 'Annie', 'Rumble'])

Teams with Ekko in the Platinum division.

In [ ]:
get_itemsets(plat, ['Ekko'])

# A brief look into the difference between Platinum and Challenger

In this section we wanted to see if there were any discrepancies between Platinum (top 17.912%) and Challenger (top .013%). Something that we found is that Challenger players tend to use a larger variety of comps than Platinum players. We looked into it more and tried to find out why this is so. 

In [ ]:
# The number of challenger compositions is about 60% greater than platinum compositions. 
print("Number of Challenger comps with a support greater than .05: " + str(len(challenger)))
print("Number of Platinum comps with a support greater than .05: " + str(len(plat)))

First, let's look at a certain combination, Dark Star. Although Dark Star is a pretty common Origin in the top compositions (Performance-wise, seen from guides and top players) this particular pairing isn't very common as there is not much overlap between infiltrators and snipers. Let's give the pairing to both platinum and challenger players and see what happens. 

Teams with both Shaco and Jhin in the Platinum division.

In [ ]:
get_itemsets(plat, ['Shaco', "Jhin"])

Teams with both Shaco and Jhin in the Challenger division.

In [ ]:
get_itemsets(challenger, ["Shaco", "Jhin"])

From what we see above, we can see that some Challenger players (support > .05) have used this pairing to win, usually making a comp out of the Dark Star trait. Although the Jhin and Shaco pairing is by no means the strongest pairing, there are some strong enough comps to come out of this pairing for Challenger players to make use of it, but this doesn't explain why there are some Challenger comps that can win with it but no Platinum comps that can. We think that this is because there's one very important aspect of the game that we didn't mention before: the champion pool is shared between all players.  This means that if all 8 players are looking for a certain champion, as more and more people grab it, there's a smaller and smaller chance of finding it. At the platinum level, it's still okay to go for the strongest compositions, but, at the highest level, players will always stop you from getting the most optimal comp. Because of this, Challenger players are very adaptable and can make a weaker composition viable, given the circumstances. Let's see this even more with a very strong pairing: Jinx and Graves. Both of these champions are part of a trait called Blasters, a strong trait that a lot of viable compositions are built from. 

In [ ]:
get_itemsets(plat, ['Jinx', "Graves"])

In [ ]:
get_itemsets(challenger, ["Jinx", "Graves"])

Again what we see here is that Challenger players make 50% more comps from blasters than Platinum players (support > .05). This further shows the adaptability of best of the best players. 